In [ ]:
import numpy as np
import random

# Función Softmax

In [ ]:
def softmax(x):
    orig_shape = x.shape
    if len(x.shape) > 1:
        c = -1*np.amax(x,axis=1,keepdims=True ) 
        x_exp = np.exp(x+c)
        x_sum = np.sum(x_exp, axis=1, keepdims=True)
        x = x_exp/x_sum    
    else:
        c = -np.amax(x)        
        x_exp = np.exp(x+c)
        x_sum = np.sum(x_exp, axis=0, keepdims=True)
        x = x_exp/x_sum
        
    assert x.shape == orig_shape
    return x

In [ ]:
def test_softmax():
    print(">>> Comenzando pruebas de softmax")
    test1 = softmax(np.array([1,2]))
    print(test1)
    ans1 = np.array([0.26894142,  0.73105858])
    assert np.allclose(test1, ans1, rtol=1e-05, atol=1e-06)

    #test2 = softmax(np.array([[1001,1002],[3,4]]))
    test2 = softmax(np.array([[1001,1002],[3,4]]))
    print(test2)
    ans2 = np.array([
        [0.26894142, 0.73105858],
        [0.26894142, 0.73105858]])
    assert np.allclose(test2, ans2, rtol=1e-05, atol=1e-06)

    test3 = softmax(np.array([[-1001,-1002]]))
    print(test3)
    ans3 = np.array([0.73105858, 0.26894142])
    assert np.allclose(test3, ans3, rtol=1e-05, atol=1e-06)
        
    test4 = softmax(np.array([[9,2,5,0,0],[7,5,0,0,0]]))
    ans4 = np.array([[9.80897665e-01 ,8.94462891e-04, 1.79657674e-02, 1.21052389e-04, 1.21052389e-04],[8.78679856e-01, 1.18916387e-01, 8.01252314e-04, 8.01252314e-04, 8.01252314e-04]])
    assert np.allclose(test4, ans4, rtol=1e-05, atol=1e-06)
    
    print(">>> Todas las prueabas fueron exitosas")

In [61]:
test_softmax()

>>> Comenzando pruebas de softmax
[0.26894142 0.73105858]
[[0.26894142 0.73105858]
 [0.26894142 0.73105858]]
[[0.73105858 0.26894142]]
>>> Todas las prueabas fueron exitosas


# Función Sigmoide

In [ ]:
def sigmoid(x):
    s = 1/(1+np.exp(-x))
    return s

In [ ]:
def sigmoid_grad(s):
    ds = s * (1-s)
    return ds

In [ ]:
def test_sigmoid():
    print(">>> Comenzando pruebas de sigmoide")
    x = np.array([[1, 2], [-1, -2]])
    f = sigmoid(x)
    g = sigmoid_grad(f)
    print(f)
    f_ans = np.array([
        [0.73105858, 0.88079708],
        [0.26894142, 0.11920292]])
    assert np.allclose(f, f_ans, rtol=1e-05, atol=1e-06)
    print(g)
    g_ans = np.array([
        [0.19661193, 0.10499359],
        [0.19661193, 0.10499359]])
    assert np.allclose(g, g_ans, rtol=1e-05, atol=1e-06)
    
    x = np.array([1,2,3])
    f = sigmoid(x)
    g = sigmoid_grad(f)
    print(f)
    f_ans = np.array([ 0.73105858, 0.88079708, 0.95257413])
    assert np.allclose(f, f_ans, rtol=1e-05, atol=1e-06)
    print(g)
    g_ans = np.array([ 0.19661193, 0.10499359, 0.04517666])
    assert np.allclose(g,g_ans, rtol=1e-05, atol=1e-06)
    
    print(">>> Todas las pruebas fueron exitosas")

In [65]:
test_sigmoid()

>>> Comenzando pruebas de sigmoide
[[0.73105858 0.88079708]
 [0.26894142 0.11920292]]
[[0.19661193 0.10499359]
 [0.19661193 0.10499359]]
[0.73105858 0.88079708 0.95257413]
[0.19661193 0.10499359 0.04517666]
>>> Todas las pruebas fueron exitosas


# Gradiente Checking

In [ ]:
def gradient_checking(f, x):

    rndstate = random.getstate()
    random.setstate(rndstate)    
    print("parametros " + str(x.shape))
    fx, grad = f(x) # Evaluate function value at original point
    h = 1e-4        # Do not change this!
    print("checking parametros " + str(x.shape))
    # Iterate over all indexes in x
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        ix = it.multi_index

        print("index " + str(ix))
        qbefore = x[ix]
        qplus = x[ix] + h
        qminus = x[ix] - h        
        x[ix] = qplus                 

        fplus, _ = f(x)        
        x[ix] = qminus
        
        fminus, _ = f(x)
        
        print("valor de fplus " + str(fplus))
        print("valor de fminus " + str(fminus))

        x[ix] = qbefore

        print("valor de fplus " + str(fplus))
        print("valor de fminus " + str(fminus))
        numgrad = (fplus-fminus)/(2*h)

        reldiff = abs(numgrad - grad[ix]) / max(1, abs(numgrad), abs(grad[ix]))
        if reldiff > 1e-5:
            print("Gradient checking fallo.")
            return

        it.iternext() # siguiente dimensión

    print("Gradient checking exitoso!\n\n")

In [ ]:
def test_gradient_checking():
    quad = lambda x: (np.sum(x ** 2), x * 2)
    #quad = lambda x: (x ** 2, x * 2)

    print("Comenzando pruebas de gradient checking...")
    gradient_checking(quad, np.array(123.456))      # scalar test
    gradient_checking(quad, np.random.randn(3,))    # 1-D test
    gradient_checking(quad, np.random.randn(4,5))   # 2-D test
    
    quad = lambda x:( 1/(1+np.exp(-x)), (1/(1+np.exp(-x)))*(1 -(1/(1+np.exp(-x)))))   
    gradient_checking(quad, np.array([1.0]))

In [68]:
test_gradient_checking()

Comenzando pruebas de gradient checking...
parametros ()
checking parametros ()
index ()
valor de fplus 15241.408627210001
valor de fminus 15241.35924481
valor de fplus 15241.408627210001
valor de fminus 15241.35924481
Gradient checking exitoso!


parametros (3,)
checking parametros (3,)
index (0,)
valor de fplus 5.573343434627016
valor de fminus 5.572725532396739
valor de fplus 5.573343434627016
valor de fminus 5.572725532396739
index (1,)
valor de fplus 5.5727997889765195
valor de fminus 5.573269178047235
valor de fplus 5.5727997889765195
valor de fminus 5.573269178047235
index (2,)
valor de fplus 5.573303535667543
valor de fminus 5.572765431356212
valor de fplus 5.573303535667543
valor de fminus 5.572765431356212
Gradient checking exitoso!


parametros (4, 5)
checking parametros (4, 5)
index (0, 0)
valor de fplus 27.417890170292488
valor de fminus 27.41752176609156
valor de fplus 27.417890170292488
valor de fminus 27.41752176609156
index (0, 1)
valor de fplus 27.417776413679007
valo

# Forward & Backward Propagation

In [ ]:
def forward_backward_prop(data, labels, params, dimensions):
    ofs = 0
    Dx, H, Dy = (dimensions[0], dimensions[1], dimensions[2])

    W1 = np.reshape(params[ofs:ofs+ Dx * H], (Dx, H))
    ofs += Dx * H
    b1 = np.reshape(params[ofs:ofs + H], (1, H))
    ofs += H
    W2 = np.reshape(params[ofs:ofs + H * Dy], (H, Dy))
    ofs += H * Dy
    b2 = np.reshape(params[ofs:ofs + Dy], (1, Dy))
         
    # forward prppagation
    z1 = np.dot(data,W1) + b1    
    h = sigmoid(z1)
    z2 = np.dot(h,W2) + b2    
    y = softmax(z2)
    cost = -1*np.sum(np.multiply(labels, np.log(y)))
    
    # backward propagation
    m = data.shape[0]
    print("data shape" + str(m))
    gradZ2 = y - labels
    gradW2 = np.dot(h.T, gradZ2)
    gradb2 = np.sum(gradZ2, axis=0, keepdims=True)  
    gradZ1 = np.dot(gradZ2, W2.T) * sigmoid_grad(h)
    gradW1 = np.dot(data.T, gradZ1)
    gradb1 = np.sum(gradZ1, axis=0, keepdims=True)
    
    
    grad = np.concatenate((gradW1.flatten(), gradb1.flatten(),
    gradW2.flatten(), gradb2.flatten()))

    print("gradientes " + str(grad))

    return cost, grad    

In [ ]:
def test_neural_network():
    print("Comenzando prueba de red neuronal")

    N = 20
    dimensions = [10, 5, 10]
    data = np.random.randn(N, dimensions[0])   # each row will be a datum
    labels = np.zeros((N, dimensions[2]))
    for i in range(N):
        labels[i, random.randint(0,dimensions[2]-1)] = 1

    params = np.random.randn((dimensions[0] + 1) * dimensions[1] + (
        dimensions[1] + 1) * dimensions[2], )

    print("Shape" + str(params.shape))
    

    gradient_checking(lambda params: forward_backward_prop(data, labels, params, dimensions), params)

In [71]:
test_neural_network()

Streaming output truncated to the last 5000 lines.
 -9.05456195e-02 -1.42204239e-01 -4.89355136e-01 -8.94176444e-01
  3.61011753e-01  6.33708363e-01 -5.48700586e-01 -5.59931355e-01
 -1.25674345e+00 -6.49127937e-01 -7.51378149e-02 -1.27376375e+00
  4.90506510e-01  3.10478797e-01  1.43396724e-01  9.43674500e-02
  9.27399125e-01  2.05161499e-01 -7.12108588e-02 -1.04564300e+00
 -6.36786514e-01  2.43313751e-01  7.26850871e-01  7.66562644e-01
  7.41062873e-01  1.24272847e+00  6.89118003e-01 -1.43681888e+00
 -8.53658117e-01  1.70910448e-01  1.80591059e-01 -1.20930046e+00
  6.39638917e+00 -7.35078607e-01 -1.37508282e+00 -4.37383578e-01
 -7.00568220e-01 -7.74170041e-01 -2.11617129e-01  1.69067885e-01
  1.41442171e-01 -1.24351681e+00  5.36159214e+00 -2.47932705e-01
 -2.65337723e+00 -6.28448462e-01  8.69601786e-02 -1.77105065e+00
 -7.94496789e-01  1.59887782e-01  1.79157929e-01 -8.22798417e-01
  6.60806298e+00 -1.10369527e+00 -1.25339363e+00 -4.89085866e-01
 -7.12588067e-01 -7.04619584e-01 -1.739